In [ ]:
import tensorflow as tf
import cv2
import os 
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
img_array = cv2.imread("Training/3/Training_1206.jpg")

In [ ]:
img_array.shape

In [ ]:
plt.imshow(img_array)

In [ ]:
Datadirectory = "Training/" #the new file for training

In [ ]:
Classes = ["0", "1", "2", "3", "4", "5", "6",]

In [ ]:
# to read all the images
for category in Classes:
    path = os.path.join(Datadirectory, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
        plt.show()
        break
    break

In [ ]:
img_size = 224 # for transfer learning
new_array = cv2.resize(img_array, (img_size, img_size))
plt.imshow(cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
new_array.shape

In [ ]:
#read all images and convert them to array

training_Data = [] # data array

def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category) #01, ## Label
        for img in os.listdir(path): 
            try:
                img_array = cv2.imread(os.path. join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass

In [ ]:
create_training_Data()

In [ ]:
print(len(training_Data))

In [ ]:
#to shuffle the sequence of the pics for trainig

import random 
random.shuffle(training_Data)

In [ ]:
X = [] #data / features
y = [] # label

for features, label in training_Data:
    X.append(features) # this one contains the array(images)
    y.append(label) # this one contains the labels

X = np.array(X).reshape(-1, img_size, img_size, 3) #converting to 4 dimension for library parameters purposes

In [ ]:
X.shape

In [ ]:
X = X/255.0;

In [ ]:
y[0]

In [ ]:
type(y)

In [ ]:
Y = np.array(y)

In [ ]:
Y.shape

# deep learning training - Transfer Learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 

In [ ]:
model = tf.keras.applications.MobileNetV2()

In [ ]:
model.summary()

# Transfer Learning - Tuning, weights will start from last chech point

In [ ]:
base_input = model.input # input

In [ ]:
base_input

In [ ]:
base_output = model.layers[-2].output  #this is to ignore the last layer because we have 7 classes not 1000

In [ ]:
base_output

In [ ]:
final_output = layers.Dense(128)(base_output) # adding new layer after the global pooling layer
final_output = layers.Activation("relu")(final_output) # activation function
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation("relu")(final_output)
final_output = layers.Dense(7, activation = "softmax")(final_output) # I have 7 classes , I used sofmatx because this is my classification layer

In [ ]:
final_output # output

In [ ]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
new_model.fit(X, Y, epochs = 25)

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
frame = cv2.imread("happy.jpg")

In [ ]:
frame.shape

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [ ]:
gray.shape

In [ ]:
faces = faceCascade.detectMultiScale(gray, 1.1, 4)
for x,y,w,h in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if len(facess) == 0:
        print("face not detected")
    else:
        for (ex,ey,ew,eh) in facess:
            face_roi = roi_color[ey: ey+eh, ex:ex+ew]

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
plt.imshow(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))

In [ ]:
final_image = cv2.resize(face_roi, (224, 224))
final_image = np.expand_dims(final_image, axis = 0)
final_image = final_image / 255.0

In [ ]:
predictions = new_model.predict(final_image)

In [ ]:
predictions

In [ ]:
np.argmax(predictions)